# 📘 Agentic Architectures 7: Blackboard Systems

Welcome to the seventh notebook in our series on agentic architectures. Today, we explore the **Blackboard System**, a powerful and highly flexible pattern for coordinating multiple specialist agents. This architecture is modeled on the idea of a group of human experts collaborating around a physical blackboard to solve a complex problem.

Instead of a rigid, predefined sequence of agent handoffs, a Blackboard system features a central, shared data store (the 'blackboard') where agents can read the current state of the problem and write their contributions. A dynamic **Controller** observes the blackboard and decides which specialist agent to activate next based on what is needed to move the solution forward. This allows for an opportunistic and emergent workflow.

To highlight its unique advantages, we will compare it to the **sequential multi-agent system** we built previously. We will present both systems with a complex financial query where the optimal path isn't a simple A → B → C sequence. We will demonstrate how the rigid sequential agent follows a suboptimal path, while the Blackboard system's dynamic Controller activates agents in a more logical, data-driven order, resulting in a more efficient and coherent analysis.

### Definition
A **Blackboard System** is a multi-agent architecture where multiple specialist agents collaborate by reading from and writing to a shared, central data repository called the 'blackboard'. A controller or scheduler dynamically determines which agent should act next based on the evolving state of the solution on the blackboard.

### High-level Workflow

1.  **Shared Memory (The Blackboard):** A central data structure holds the current state of the problem, including the user's request, intermediate findings, and partial solutions.
2.  **Specialist Agents:** A pool of independent agents, each with a specific expertise, continuously monitors the blackboard.
3.  **Controller:** A central 'controller' agent also monitors the blackboard. Its job is to analyze the current state and decide which specialist agent is best equipped to make the next contribution.
4.  **Opportunistic Activation:** The Controller activates the chosen agent. The agent reads the relevant data from the blackboard, performs its task, and writes its findings back to the blackboard.
5.  **Iteration:** The process repeats, with the Controller activating different agents in a dynamic sequence, until it determines that the solution on the blackboard is complete.

### When to Use / Applications
*   **Complex, Ill-Structured Problems:** Ideal for problems where the solution path is not known in advance and requires an emergent, opportunistic strategy (e.g., complex diagnostics, scientific discovery).
*   **Multi-Modal Systems:** A great way to coordinate agents that work with different data types (text, images, code), as they can all post their findings to the shared blackboard.
*   **Dynamic Sense-Making:** Situations that require synthesizing information from many disparate, asynchronous sources.

### Strengths & Weaknesses
*   **Strengths:**
    *   **Flexibility & Adaptability:** The workflow is not hardcoded; it emerges based on the problem, making the system highly adaptive.
    *   **Modularity:** It's very easy to add or remove specialist agents without re-architecting the entire system.
*   **Weaknesses:**
    *   **Controller Complexity:** The intelligence of the entire system depends heavily on the sophistication of the Controller. A naive Controller can lead to inefficient or looping behavior.
    *   **Debugging Challenges:** The non-linear, emergent nature of the workflow can sometimes make it harder to trace and debug than a simple sequential process.

## Phase 0: Foundation & Setup

We'll begin with our standard setup process: installing libraries and configuring API keys for Nebius, LangSmith, and Tavily.

### Step 0.1: Installing Core Libraries

**What we are going to do:**
We will install our standard suite of libraries for this project series.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

### Step 0.2: Importing Libraries and Setting Up Keys

**What we are going to do:**
We will import the necessary modules and load our API keys from a `.env` file.

**Action Required:** Create a `.env` file in this directory with your keys:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [2]:
import os
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# LangChain components
from langchain_nebius import ChatNebius
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Blackboard (Nebius)"

for key in ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} not found. Please create a .env file and set it.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1: The Baseline - A Sequential Multi-Agent System (Corrected)

To understand the blackboard's flexibility, we first need a correctly functioning sequential system. The original version failed because specialists didn't use the output of previous steps. We will correct this by ensuring each agent receives the necessary context from the state.

### Step 1.1: Building the (Corrected) Sequential Team

**What we are going to do:**
We will define specialist agents that explicitly use the outputs of their predecessors, then wire them in a fixed, linear sequence.

In [3]:
console = Console()
# Using a more capable model to handle complex instructions better
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
search_tool = TavilySearch(max_results=2)

# State for the sequential agent
class SequentialState(TypedDict):
    user_request: str
    news_report: Optional[str]
    technical_report: Optional[str]
    financial_report: Optional[str]
    final_report: Optional[str]

# --- CORRECTED SPECIALIST NODES FOR SEQUENTIAL AGENT ---
# The key change is that each agent now gets context from previous steps, not just the original request.

def news_analyst_node_seq(state: SequentialState):
    console.print("--- (Sequential) CALLING NEWS ANALYST ---")
    prompt = f"Your task is to act as an expert News Analyst. Find the latest major news about the topic in the user's request and provide a concise summary.\n\nUser Request: {state['user_request']}"
    agent = llm.bind_tools([search_tool])
    result = agent.invoke(prompt)
    return {"news_report": result.content}

def technical_analyst_node_seq(state: SequentialState):
    console.print("--- (Sequential) CALLING TECHNICAL ANALYST ---")
    # This agent now uses the news report as context.
    prompt = f"Your task is to act as an expert Technical Analyst. Based on the following news report, conduct a technical analysis of the company's stock.\n\nNews Report:\n{state['news_report']}"
    agent = llm.bind_tools([search_tool])
    result = agent.invoke(prompt)
    return {"technical_report": result.content}

def financial_analyst_node_seq(state: SequentialState):
    console.print("--- (Sequential) CALLING FINANCIAL ANALYST ---")
    # This agent also uses the news report as context.
    prompt = f"Your task is to act as an expert Financial Analyst. Based on the following news report, analyze the company's recent financial performance.\n\nNews Report:\n{state['news_report']}"
    agent = llm.bind_tools([search_tool])
    result = agent.invoke(prompt)
    return {"financial_report": result.content}


def report_writer_node_seq(state: SequentialState):
    console.print("--- (Sequential) CALLING REPORT WRITER ---")
    prompt = f"""You are an expert report writer. Your task is to synthesize the information from the News, Technical, and Financial analysts into a single, cohesive report that directly answers the user's original request.

User Request: {state['user_request']}

Here are the reports to combine:
---
News Report: {state['news_report']}
---
Technical Report: {state['technical_report']}
---
Financial Report: {state['financial_report']}
"""
    report = llm.invoke(prompt).content
    return {"final_report": report}

# Build the sequential graph
seq_graph_builder = StateGraph(SequentialState)
seq_graph_builder.add_node("news", news_analyst_node_seq)
seq_graph_builder.add_node("tech", technical_analyst_node_seq)
seq_graph_builder.add_node("finance", financial_analyst_node_seq)
seq_graph_builder.add_node("writer", report_writer_node_seq)

# The rigid, hardcoded sequence
seq_graph_builder.set_entry_point("news")
seq_graph_builder.add_edge("news", "tech")
seq_graph_builder.add_edge("tech", "finance")
seq_graph_builder.add_edge("finance", "writer")
seq_graph_builder.add_edge("writer", END)

sequential_app = seq_graph_builder.compile()
print("Corrected sequential multi-agent system compiled successfully.")

Corrected sequential multi-agent system compiled successfully.


### Step 1.2: Testing the Sequential Agent on a Dynamic Problem

Now that the sequential agent is correctly passing context, let's observe its behavior. It will produce a more coherent report, but its *process* will still be inefficient and fail to follow the conditional logic.

In [4]:
dynamic_query = "Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative)."

console.print(f"[bold yellow]Testing CORRECTED SEQUENTIAL agent on a dynamic query:[/bold yellow]\n'{dynamic_query}'\n")

# Run the graph
final_seq_output = sequential_app.invoke({"user_request": dynamic_query})

console.print("\n--- [bold red]Final Report from Sequential Agent[/bold red] ---")
console.print(Markdown(final_seq_output['final_report']))

Testing CORRECTED SEQUENTIAL agent on a dynamic query:
'Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).'


--- (Sequential) CALLING NEWS ANALYST ---
--- (Sequential) CALLING TECHNICAL ANALYST ---
--- (Sequential) CALLING FINANCIAL ANALYST ---
--- (Sequential) CALLING REPORT WRITER ---



--- Final Report from Sequential Agent ---


### Comprehensive Analysis of Nvidia Following Latest News

**User Request:** Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).

**1. News Summary:**
Recent major news concerning Nvidia revolves around the announcement of their next-generation AI chip architecture, codenamed "Rubin." This news has generated significant positive sentiment in the market, with analysts highlighting the potential for continued dominance in the AI hardware space. The announcement points to a roadmap extending beyond the current "Blackwell" chips, suggesting sustained innovation and growth. This news is overwhelmingly positive.

**2. Technical Analysis:**
Following the positive news, Nvidia's stock (NVDA) has shown strong bullish indicators. The stock price has broken through previous resistance levels and is trading well above its 50-day and 200-day moving averages. Trading volume has been high on up-days, confirming the bullish trend. Key indicators like the Relative Strength Index (RSI) are in overbought territory, suggesting a possible short-term pullback, but the overall momentum remains strongly positive.

**3. Financial Analysis:**
Nvidia's recent financial performance has been exceptionally strong, driven by surging demand for its data center GPUs. The company's latest quarterly earnings report showed revenue and net income figures that significantly beat analyst expectations. Gross margins have expanded, and the company has provided a very optimistic forecast for the upcoming quarter, further bolstering investor confidence.

**Conclusion:**
The combination of groundbreaking product announcements, positive market sentiment, strong technical stock performance, and record-breaking financial results paints a very bullish picture for Nvidia. While the user's request specified an 'either/or' analysis based on sentiment, this report includes all analyses for completeness, confirming the company's robust position from multiple perspectives.

**Discussion of the (Corrected) Output:**
The agent now produces a full, logical report. However, the execution trace `News → Technical → Financial` reveals its fundamental flaw. It performed **both** a technical and a financial analysis, completely ignoring the user's conditional request ("either... or..."). This is inefficient and demonstrates the rigidity we aim to solve with the Blackboard architecture.

## Phase 2: The Advanced Approach - A Blackboard System (Corrected)

Now, we'll build the Blackboard system. The key to fixing the original's looping behavior is to craft a much more intelligent prompt for the **Controller**, making it aware of its role as a stateful planner.

### Step 2.1: Defining the Blackboard and the (Corrected) Controller

**What we are going to do:**
1.  **Blackboard State:** Define a `BlackboardState` for shared memory.
2.  **Specialist Agents:** Define the specialist nodes. They will be similar to our previous agents.
3.  **Controller (Corrected):** Create a robust `controller_node` with a prompt that explicitly reasons about completed steps and remaining goals. This is the most critical change.

In [5]:
# The Blackboard State holds all information
class BlackboardState(TypedDict):
    user_request: str
    # The central blackboard where agents post their findings as strings
    blackboard: List[str]
    # List of available agents for the controller to choose from
    available_agents: List[str]
    # The controller's next decision
    next_agent: Optional[str]

# Pydantic model for the Controller's decision
# CORRECTION: Added the list of available agents to the field description to guide the LLM's choice.
class ControllerDecision(BaseModel):
    next_agent: str = Field(description="The name of the next agent to call. Must be one of ['News Analyst', 'Technical Analyst', 'Financial Analyst', 'Report Writer'] or 'FINISH'.")
    reasoning: str = Field(description="A brief reason for choosing the next agent.")

# Reusable factory for creating specialist agents for the blackboard
def create_blackboard_specialist(persona: str, agent_name: str):
    system_prompt = f"""You are an expert specialist agent: a {persona}.
Your task is to contribute to a larger goal by performing your specific function.
Read the initial User Request and the current Blackboard for context.
Use your tools to find the required information.
Finally, post your concise markdown report back to the blackboard. Your report should be signed with your name '{agent_name}'.
"""
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "User Request: {user_request}\n\nBlackboard (previous reports):\n{blackboard_str}")
    ])
    agent = prompt_template | llm.bind_tools([search_tool])

    def specialist_node(state: BlackboardState):
        console.print(f"--- (Blackboard) AGENT '{agent_name}' is working... ---")
        blackboard_str = "\n---\n".join(state["blackboard"])
        result = agent.invoke({"user_request": state["user_request"], "blackboard_str": blackboard_str})
        report = f"**Report from {agent_name}:**\n{result.content}"
        # Append the new report to the list of blackboard entries
        return {"blackboard": state["blackboard"] + [report]}
    return specialist_node

# Create the specialist agent nodes
news_analyst_bb = create_blackboard_specialist("News Analyst", "News Analyst")
technical_analyst_bb = create_blackboard_specialist("Technical Analyst", "Technical Analyst")
financial_analyst_bb = create_blackboard_specialist("Financial Analyst", "Financial Analyst")
report_writer_bb = create_blackboard_specialist("Report Writer who synthesizes a final answer from the blackboard", "Report Writer")

# --- THE CORRECTED, INTELLIGENT CONTROLLER NODE ---
# This is the most important fix. The prompt is now much more sophisticated.
def controller_node(state: BlackboardState):
    console.print("--- CONTROLLER: Analyzing blackboard... ---")

    # Use a structured output LLM to make the decision
    controller_llm = llm.with_structured_output(ControllerDecision)

    blackboard_content = "\n\n".join(state['blackboard'])
    agent_list = state['available_agents']

    # The new prompt is state-aware and goal-oriented.
    prompt = f"""You are the central controller of a multi-agent system. Your job is to analyze the shared blackboard and the original user request to decide which specialist agent should run next.

**Original User Request:**
{state['user_request']}

**Current Blackboard Content:**
---
{blackboard_content if blackboard_content else "The blackboard is currently empty."}
---

**Available Specialist Agents:**
{', '.join(agent_list)}

**Your Task:**
1.  Read the user request and the current blackboard content carefully.
2.  Determine what the *next logical step* is to move closer to a complete answer.
3.  Choose the single best agent to perform that step from the list of available agents.
4.  If the user's request has been fully addressed and a final report has been written, choose 'FINISH'. Do not finish until a "Report Writer" has provided a final, synthesized answer.

Provide your decision in the required format.
"""
    decision_result = controller_llm.invoke(prompt)
    console.print(f"--- CONTROLLER: Decision is to call '{decision_result.next_agent}'. Reason: {decision_result.reasoning} ---")

    # The dictionary returned here updates the 'next_agent' key in the graph's state
    return {"next_agent": decision_result.next_agent}

print("Blackboard components and corrected Controller node defined.")

Blackboard components and corrected Controller node defined.


### Step 2.2: Building the Blackboard Graph

Now we wire the components into a dynamic graph. The Controller acts as a central router. After any specialist runs, control always returns to the Controller to decide the next step.

In [6]:
bb_graph_builder = StateGraph(BlackboardState)

# Add all nodes to the graph
bb_graph_builder.add_node("Controller", controller_node)
bb_graph_builder.add_node("News Analyst", news_analyst_bb)
bb_graph_builder.add_node("Technical Analyst", technical_analyst_bb)
bb_graph_builder.add_node("Financial Analyst", financial_analyst_bb)
bb_graph_builder.add_node("Report Writer", report_writer_bb)

bb_graph_builder.set_entry_point("Controller")

# This function defines the dynamic routing logic based on the Controller's decision
def route_to_agent(state: BlackboardState):
    return state["next_agent"]

# The conditional edges route from the Controller to the chosen specialist or to the end
bb_graph_builder.add_conditional_edges(
    "Controller",
    route_to_agent,
    {
        "News Analyst": "News Analyst",
        "Technical Analyst": "Technical Analyst",
        "Financial Analyst": "Financial Analyst",
        "Report Writer": "Report Writer",
        "FINISH": END
    }
)

# After any specialist runs, control always returns to the Controller for the next decision
bb_graph_builder.add_edge("News Analyst", "Controller")
bb_graph_builder.add_edge("Technical Analyst", "Controller")
bb_graph_builder.add_edge("Financial Analyst", "Controller")
bb_graph_builder.add_edge("Report Writer", "Controller")

blackboard_app = bb_graph_builder.compile()
print("Blackboard system compiled successfully.")

Blackboard system compiled successfully.


## Phase 3: Head-to-Head Comparison

Let's run our new Blackboard system on the same dynamic task and observe its intelligent workflow.

In [7]:
console.print(f"[bold green]Testing BLACKBOARD system on the same dynamic query:[/bold green]\n'{dynamic_query}'\n")

agent_list = ["News Analyst", "Technical Analyst", "Financial Analyst", "Report Writer"]
initial_bb_input = {"user_request": dynamic_query, "blackboard": [], "available_agents": agent_list}

# We use stream to observe the step-by-step process
final_bb_output = None
for chunk in blackboard_app.stream(initial_bb_input, {"recursion_limit": 10}):
    final_bb_output = chunk
    console.print("\n--- [bold purple]Current Blackboard State[/bold purple] ---")
    # Pretty print each report on the blackboard
    for i, report in enumerate(final_bb_output.get('blackboard', [])):
        console.print(f"--- Report {i+1} ---")
        console.print(Markdown(report))
    console.print("\n")

console.print("\n--- [bold green]Final Report from Blackboard System[/bold green] ---")
# The final report is the last item posted to the blackboard by the writer
final_report_content = final_bb_output['blackboard'][-1]
console.print(Markdown(final_report_content))

Testing BLACKBOARD system on the same dynamic query:
'Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).'


--- CONTROLLER: Analyzing blackboard... ---
--- CONTROLLER: Decision is to call 'News Analyst'. Reason: The blackboard is empty, so the first step is to gather the latest news about Nvidia as requested by the user. ---



--- Current Blackboard State ---



--- (Blackboard) AGENT 'News Analyst' is working... ---
--- CONTROLLER: Analyzing blackboard... ---
--- CONTROLLER: Decision is to call 'Technical Analyst'. Reason: The news report on the blackboard is positive, mentioning a new chip architecture and market dominance. According to the user's request, a technical analysis is the correct next step for positive news. ---



--- Current Blackboard State ---


...

**Report from Technical Analyst:**
Based on the positive news regarding Nvidia's accelerated roadmap, a technical analysis of NVDA stock shows strong bullish momentum. The stock is currently in a clear uptrend, trading significantly above its 50-day and 200-day simple moving averages (SMAs). The announcement created a bullish gap on the price chart, which has held, indicating strong buying pressure. The Relative Strength Index (RSI) is high, suggesting the stock is in overbought territory, but this is common for a stock in a strong uptrend. Key support is now established at the top of the recent price gap.

--- Report 3 ---


**Report from Report Writer:**
### Final Analysis of Nvidia Based on Recent News

**Initial Request:** Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).

**Process Summary:**
1.  **News Analysis:** The latest major news for Nvidia is the announcement of its next-generation "Rubin" AI chip platform, which has generated highly positive market sentiment.
2.  **Conditional Action:** As per the user's request, the positive news sentiment triggered a technical analysis.
3.  **Technical Analysis:** The technical outlook for NVDA stock is strongly bullish. The stock is in a clear uptrend, trading above key moving averages, with significant buying pressure confirmed by a recent price gap.

**Conclusion:** Following the user's instructions, an analysis of recent positive news about Nvidia's accelerated innovation led to a technical review, which confirms a strong bullish momentum for the company's stock.


--- Final Report from Blackboard System ---


**Report from Report Writer:**
### Final Analysis of Nvidia Based on Recent News

**Initial Request:** Find the latest major news about Nvidia. Based on the sentiment of that news, conduct either a technical analysis (if the news is neutral or positive) or a financial analysis of their recent performance (if the news is negative).

**Process Summary:**
1.  **News Analysis:** The latest major news for Nvidia is the announcement of its next-generation "Rubin" AI chip platform, which has generated highly positive market sentiment.
2.  **Conditional Action:** As per the user's request, the positive news sentiment triggered a technical analysis.
3.  **Technical Analysis:** The technical outlook for NVDA stock is strongly bullish. The stock is in a clear uptrend, trading above key moving averages, with significant buying pressure confirmed by a recent price gap.

**Conclusion:** Following the user's instructions, an analysis of recent positive news about Nvidia's accelerated innovation led to a technical review, which confirms a strong bullish momentum for the company's stock.

**Discussion of the (Corrected) Output:**
Success! The `GraphRecursionError` is gone. The execution trace reveals a far more intelligent process:

1.  **Controller Start:** The Controller starts and, seeing an empty blackboard, correctly decides to call the **News Analyst** first.
2.  **News Analyst Runs:** The News Analyst finds the latest news and posts its report to the blackboard.
3.  **Controller Re-evaluates:** Control returns to the Controller. It reads the News Analyst's report, understands the sentiment, and follows the user's logic. It intelligently decides to call the appropriate next analyst (**Technical** or **Financial**), completely skipping the other one.
4.  **Specialist Runs:** The chosen analyst performs its task and adds its report to the blackboard.
5.  **Controller Finishes:** The Controller sees all necessary analysis is complete and calls the **Report Writer** to synthesize the final answer.
6.  **Final Call:** After the writer posts the final report, the controller sees this and decides to **FINISH**.

This dynamic, opportunistic workflow is the hallmark of a properly functioning Blackboard system. It perfectly followed the user's complex conditional logic, saving time and resources.

## Phase 4: Quantitative Evaluation

To formalize the comparison, we will use an LLM-as-a-Judge to score both systems on instruction following and process efficiency.

In [8]:
class ProcessLogicEvaluation(BaseModel):
    """Schema for evaluating an agent's logical process."""
    instruction_following_score: int = Field(description="Score 1-10 on how well the agent followed the user's specific conditional instructions (e.g., the 'either/or' logic).")
    process_efficiency_score: int = Field(description="Score 1-10 on whether the agent took the most direct path and avoided unnecessary work.")
    justification: str = Field(description="A brief justification for the scores, referencing specific steps the agent took.")

# Use a strong model for judging
judge_llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0).with_structured_output(ProcessLogicEvaluation)

def evaluate_agent_logic(query: str, final_state: dict):
    # Reconstruct a simplified trace for the judge
    trace = ""
    agent_type = "Unknown"
    if 'blackboard' in final_state: # Blackboard agent
        agent_type = "Blackboard"
        trace = "\n---\n".join(final_state['blackboard'])
    else: # Sequential agent
        agent_type = "Sequential"
        trace = f"1. News Report Generated: {final_state.get('news_report')}\n---\n2. Technical Report Generated: {final_state.get('technical_report')}\n---\n3. Financial Report Generated: {final_state.get('financial_report')}"

    prompt = f"""You are an expert judge of AI agent processes. Your task is to evaluate an agent's performance based on its generated content trace.

**User's Original Task:**
"{query}"

**Agent's Type:** {agent_type}
**Agent's Generated Content Trace:**
```
{trace}
```

**Evaluation Criteria:**
1.  **Instruction Following:** Did the agent respect the conditional logic in the user's task? (e.g., "either a technical analysis... or a financial analysis"). A high score means it followed the logic perfectly. A low score means it ignored it.
2.  **Process Efficiency:** Did the agent avoid doing unnecessary work? A high score means it only ran the required specialists. A low score means it ran specialists that the user's logic explicitly said to skip.

Based on the trace, provide your evaluation.
"""
    return judge_llm.invoke(prompt)

console.print("--- [bold]Evaluating Sequential Agent's Process[/bold] ---")
seq_agent_evaluation = evaluate_agent_logic(dynamic_query, final_seq_output)
console.print(seq_agent_evaluation.dict())

console.print("\n--- [bold]Evaluating Blackboard System's Process[/bold] ---")
bb_agent_evaluation = evaluate_agent_logic(dynamic_query, final_bb_output)
console.print(bb_agent_evaluation.dict())

--- Evaluating Sequential Agent's Process ---


{'instruction_following_score': 2, 'process_efficiency_score': 3, 'justification': "The agent failed to follow the core conditional instruction ('either/or'). Instead of choosing one path based on the news sentiment, it executed both the technical and financial analyses. This shows a lack of adherence to the user's logic and resulted in an inefficient process by performing unnecessary work."}



--- Evaluating Blackboard System's Process ---


{'instruction_following_score': 10, 'process_efficiency_score': 10, 'justification': 'The agent perfectly followed the user\'s conditional instructions. After the News Analyst reported positive sentiment, the system correctly chose to run the Technical Analyst and completely skipped the Financial Analyst. This demonstrates both flawless instruction following and optimal process efficiency, as no unnecessary work was performed.'}


**Discussion of the Evaluation Output:**
The judge's scores provide a clear, quantitative verdict:

- The **Sequential Agent** will receive a very low `instruction_following_score` (e.g., 2/10) because it blatantly ignored the "either/or" condition. Its `process_efficiency_score` will also be low (e.g., 3/10) because it performed an entire analysis that was explicitly not required.
- The **Blackboard System** will receive near-perfect scores (e.g., 10/10) for both. The judge will recognize that the Controller's dynamic decision-making allowed the system to follow the user's instructions precisely and to operate with maximum efficiency by only activating the necessary specialist.

This evaluation provides definitive proof that for complex, emergent problems where the path forward depends on intermediate results, the flexibility of the Blackboard architecture is vastly superior to a rigid, predefined workflow.

## Conclusion

In this notebook, we have implemented and corrected a **Blackboard System**, demonstrating its significant advantages over a sequential multi-agent architecture. By introducing a shared memory (the blackboard) and an intelligent, state-aware **Controller**, we created a system that is not just collaborative, but also adaptive and opportunistic.

The head-to-head comparison showed that for tasks with conditional logic, the Blackboard system's ability to choose the right expert at the right time leads to a more efficient and logically sound process. While it requires a more sophisticated Controller, this architecture is a powerful tool for tackling the kind of ill-structured, real-world problems that rigid, linear workflows cannot solve effectively.